In [68]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, RandomizedSearchCV, StratifiedKFold
from scipy.stats import expon, reciprocal, uniform
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, DotProduct, ExpSineSquared, RationalQuadratic
import numpy as np
from sklearn.feature_selection import RFE, SelectFromModel, RFECV
from sklearn.compose import ColumnTransformer
# from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

In [69]:
def load_data(file_list, df_activities, df_links_network):
    data_frames = []
    for file in file_list:
        with open(file, 'r') as f:
            data = json.load(f)
            if isinstance(data['link_counts'], dict):
                data['link_counts'] = data['link_counts'].values()
            df_links = pd.DataFrame({
                'link_id': data['links_id'],
                'link_from': data['link_from'],
                'link_to': data['link_to'],
                'link_length': data['link_length'],
                'link_freespeed': data['link_freespeed'],
                'link_capacity': data['link_capacity'],
                'link_permlanes': data['link_permlanes'],
                'link_counts': data['link_counts']
            })
            df_nodes = pd.DataFrame({
                'node_id': data['nodes_id'],
                'node_x': data['nodes_x'],
                'node_y': data['nodes_y']
            })
            df_od_pairs = pd.DataFrame(data['o_d_pairs'], columns=['origin', 'destination'])
            
            df_work = pd.DataFrame({
                        'work_x': data['work_x'],
                        'work_y': data['work_y'],
                        'go_to_work': data['go_to_work']
            })
            df_home = pd.DataFrame({
                'home_x': data['home_x'],
                'home_y': data['home_y'],
                'go_to_home': data['go_to_home']
            })
            
            df_links = df_links.merge(df_nodes, how='left', left_on='link_from', right_on='node_id')
            df_links = df_links.rename(columns={'node_x': 'start_node_x', 'node_y': 'start_node_y'})
            df_links.drop('node_id', axis=1, inplace=True)
            df_links = df_links.merge(df_nodes, how='left', left_on='link_to', right_on='node_id')
            df_links = df_links.rename(columns={'node_x': 'end_node_x', 'node_y': 'end_node_y'})
            df_links.drop('node_id', axis=1, inplace=True) 
            
            origin_counts = df_od_pairs['origin'].value_counts()
            df_origin_counts = origin_counts.reset_index()
            df_origin_counts.columns = ['origin', 'start_count']
            destination_counts = df_od_pairs['destination'].value_counts()
            df_destination_counts = destination_counts.reset_index()
            df_destination_counts.columns = ['destination', 'end_count']
            df_links = df_links.merge(df_origin_counts, how='left', left_on='link_from', right_on='origin')
            df_links.drop('origin', axis=1, inplace=True)
            df_links = df_links.merge(df_destination_counts, how='left', left_on='link_to', right_on='destination')
            df_links.drop('destination', axis=1, inplace=True)
            df_links[['start_count','end_count']] = df_links[['start_count','end_count']].fillna(-1)
            
            # Calculate time of go_to_work and go_to_sum
            df_act_work = df_activities[df_activities['activity_type_main']=='work'].drop(['end_time'], axis=1)
            df_act_work = df_act_work.merge(df_work, how='left', left_on=['x','y'], right_on=['work_x','work_y'])
            df_act_work.drop(['x','y'], axis=1, inplace=True)
            df_act_work_agg = df_act_work.groupby(by="link")['go_to_work'].sum().reset_index(drop=False)
            df_act_home = df_activities[df_activities['activity_type_main']=='home'].drop(['end_time'], axis=1)
            df_act_home = df_act_home.merge(df_home, how='left', left_on=['x','y'], right_on=['home_x','home_y'])
            df_act_home.drop(['x','y'], axis=1, inplace=True)
            df_act_home_agg = df_act_home.groupby(by="link")['go_to_home'].sum().reset_index(drop=False)
            df_act_agg = df_act_home_agg.merge(df_act_work_agg, how='outer', on='link')
            df_act_agg.fillna(0, inplace=True)
            df_act_agg['go_to_sum'] = df_act_agg['go_to_home'] + df_act_agg['go_to_work']

            df_rushhr = df_activities[df_activities['end_time']!=-1]
            df_rushhr.loc[:, 'rush_hour'] = 0
            df_rushhr.loc[df_rushhr['end_time'].between(pd.to_timedelta('08:00:00'), pd.to_timedelta('10:00:00'), inclusive='both'), 'rush_hour'] = 1
            df_rushhr.loc[df_rushhr['end_time'].between(pd.to_timedelta('16:00:00'), pd.to_timedelta('19:00:00'), inclusive='both'), 'rush_hour'] = 1
            df_rushhr.drop(['end_time', 'max_dur', 'zoneId', 'cemdapStopDuration_s'], axis=1, inplace=True)
            df_rushhragg = df_rushhr.groupby(by="link").sum()['rush_hour'].reset_index(drop=False)
            
            df_maxduragg = df_activities[df_activities['max_dur']!=-1].groupby(by='link')['max_dur'].sum().reset_index(drop=False)
            
            df_activities['cemdapStopDuration_s'] = df_activities['cemdapStopDuration_s'].astype(float)
            df_cemagg = df_activities[df_activities['cemdapStopDuration_s']!=-1].groupby(by='link')['cemdapStopDuration_s'].sum().reset_index(drop=False)
            
            df_temp = df_links.merge(df_links_network, how='left', on=['start_node_x','start_node_y','end_node_x','end_node_y'])
            df_temp = df_temp[['link_id_x','link_from','link_to','link_id_y','from', 'to', 'type']]
            df_temp = df_temp.merge(df_act_agg, how='left', left_on='link_id_y', right_on='link')
            df_temp.drop('link', axis=1, inplace=True)
            df_temp = df_temp.merge(df_rushhragg, how='left', left_on='link_id_y', right_on='link')
            df_temp.drop('link', axis=1, inplace=True)
            df_temp = df_temp.merge(df_maxduragg, how='left', left_on='link_id_y', right_on='link')
            df_temp.drop('link', axis=1, inplace=True)
            df_temp = df_temp.merge(df_cemagg, how='left', left_on='link_id_y', right_on='link')
            df_temp.fillna({'cemdapStopDuration_s':-1, 'max_dur':-1, 'rush_hour': -1, 'go_to_sum': -1}, inplace=True)
            df_temp = df_temp[['link_id_x', 'go_to_sum', 'rush_hour', 'max_dur', 'cemdapStopDuration_s', 'type']]
            
            df_links = df_links.merge(df_temp, how='left', left_on='link_id', right_on='link_id_x')
            df_links.drop('link_id_x', axis=1, inplace=True)
        data_frames.append(df_links)
    return pd.concat(data_frames, ignore_index=True)


In [70]:
train_files = ['s-0.json', 's-1.json', 's-2.json', 's-3.json', 's-4.json','s-5.json', 's-6.json', 's-7.json', 's-8.json', 's-9.json'] 
test_files = ['s-15.json', 's-16.json', 's-17.json', 's-18.json','s-19.json']
validate_files = ['s-10.json', 's-11.json', 's-12.json', 's-13.json','s-14.json']
train_files = ['Data/cutoutWorlds/Train/po-1_pn-1.0_sn-1/' + i for i in train_files]
test_files = ['Data/cutoutWorlds/Test/po-1_pn-1.0_sn-1/' + j for j in test_files]
validate_files = ['Data/cutoutWorlds/Validate/po-1_pn-1.0_sn-1/' + k for k in validate_files]
df_activities = pd.read_pickle("Data/cutoutWorlds/Train/po-1_pn-1.0_sn-1/df_activities.pkl")
df_links_network = pd.read_pickle("Data/cutoutWorlds/Train/po-1_pn-1.0_sn-1/df_links_network.pkl")
train_data = load_data(train_files, df_activities, df_links_network)
test_data = load_data(test_files, df_activities, df_links_network)
validate_data = load_data(validate_files, df_activities, df_links_network)
Big_train_data = pd.concat([train_data, validate_data], ignore_index=True)

C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rushhr.loc[:, 'rush_hour'] = 0
C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rushhr.drop(['end_time', 'max_dur', 'zoneId', 'cemdapStopDuration_s'], axis=1, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:73: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. E

C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rushhr.loc[:, 'rush_hour'] = 0
C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rushhr.drop(['end_time', 'max_dur', 'zoneId', 'cemdapStopDuration_s'], axis=1, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:73: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. E

C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rushhr.loc[:, 'rush_hour'] = 0
C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rushhr.drop(['end_time', 'max_dur', 'zoneId', 'cemdapStopDuration_s'], axis=1, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_11596\3992581593.py:73: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. E

In [71]:
numerical_features = ['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes', 'start_node_x', 'start_node_y', 'end_node_x', 'end_node_y', 'start_count', 'end_count', 'go_to_sum', 'rush_hour', 'max_dur', 'cemdapStopDuration_s']
category_feature = ['type']
X_t = Big_train_data.drop(columns=['link_counts'])
y_t = Big_train_data['link_counts']
# X_v = validate_data.drop(columns=['link_counts'])
# y_v = validate_data['link_counts']
X_te = test_data.drop(columns=['link_counts'])
y_te = test_data['link_counts']
scaler = StandardScaler()
le = LabelEncoder()
ohe = OneHotEncoder(sparse_output=False)
ct = ColumnTransformer(
     [("num_preprocess", scaler, numerical_features),
      ("text_preprocess", ohe, category_feature)], remainder='passthrough').set_output(transform="pandas")
X_t = ct.fit_transform(X_t)
# X_v = ct.fit_transform(X_v)  
X_te = ct.fit_transform(X_te)
# X_t[numerical_features] = scaler.fit_transform(X_t[numerical_features])
# X_v[numerical_features] = scaler.fit_transform(X_v[numerical_features])
# X_te[numerical_features] = scaler.fit_transform(X_te[numerical_features])
# X_t[category_feature] = le.fit_transform(X_t[category_feature])
# X_v[category_feature] = le.fit_transform(X_v[category_feature])
# X_te[category_feature] = le.fit_transform(X_te[category_feature])


In [73]:
# Initialize models
kf = KFold(n_splits=5, shuffle=True, random_state=42)
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': LassoCV(cv=kf, random_state=42, max_iter=100000),
    'Ridge': RidgeCV(cv=kf),
    'SVR': SVR(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Artificial Neural Network': MLPRegressor(),
#     'Gaussian Process Regression': GaussianProcessRegressor(kernel=RBF(length_scale=1.0) + WhiteKernel(noise_level=1.0), alpha=0.1, n_restarts_optimizer=3)
}


In [74]:
# Function to train and evaluate models
def evaluate_models(models, X_train, y_train):
    results = {}
    for name, model in models.items():
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         mse = mean_squared_error(y_test, y_pred)
#         mae = mean_absolute_error(y_test, y_pred)
        mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
        mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

        # Define the cross-validation strategy (e.g., 5-fold cross-validation)
        kf = KFold(n_splits=5, shuffle=True, random_state=42)

        # Perform k-fold cross-validation and calculate MSE and MAE
        mse_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mse_scorer)
        mae_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mae_scorer)

        # Display the mean MSE and MAE across folds
        mean_mse = -mse_scores.mean()
        mean_mae = -mae_scores.mean()
        std_mse = mse_scores.std()
        # mape = mean_absolute_percentage_error(y_test, y_pred)
        # r2 = r2_score(y_test, y_pred)
        print(name + " done")
        
        results[name] = {'MAE': mean_mae, 'MSE': mean_mse, 'MSE_std': std_mse}
    
    return results

# Function to train and evaluate models
def feature_select_models(models, X_train, y_train):
    mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    kf = KFold(n_splits=5, shuffle=True, random_state=42) 
    results = {}
    for name, model in models.items():
        if name not in ['SVR', 'Artificial Neural Network', 'Gaussian Process Regression']:
            selector = RFECV(model, step=1, cv=kf, scoring=mse_scorer).fit(X_train, y_train)
            print(f'{name} selection done')

        else:
            # Fit Random Forest to get feature importances
            rf = RandomForestRegressor()
            rf.fit(X_train, y_train)
            # Select features based on importances
            selector = RFECV(estimator=rf, step=1, cv=kf, scoring=mse_scorer).fit(X_train, y_train)
            print(f'{name} selection done')
            
        selected_features = X_train.columns[selector.support_]
        X_train_reduced = X_train[selected_features] 
        mse_scores = cross_val_score(model, X_train_reduced, y_train, cv=kf, scoring=mse_scorer)
        mse = -mse_scores.mean()
        mse_std = mse_scores.std()           
        mae_scores = cross_val_score(model, X_train_reduced, y_train, cv=kf, scoring=mae_scorer)
        mean_mae = -mae_scores.mean()

        results[name] = {'MAE': mean_mae, 'MSE': mse, 'MSE_std': mse_std, 'selected_feature': selected_features}

            
    return results

# Train and evaluate
results = evaluate_models(models, X_t, y_t)

# results_feature = feature_select_models(models, X_t, y_t)


Linear Regression done
Lasso done
Ridge done
SVR done
Random Forest done
Gradient Boosting done


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer

Artificial Neural Network done


C:\Users\user\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [75]:
# before hyperparametertuning
results

{'Linear Regression': {'MAE': 12.425649743866481,
  'MSE': 809.9501008981372,
  'MSE_std': 140.3128930459079},
 'Lasso': {'MAE': 12.838810988648827,
  'MSE': 888.6097269093482,
  'MSE_std': 165.55719447698084},
 'Ridge': {'MAE': 12.397992690840534,
  'MSE': 810.8172874182757,
  'MSE_std': 142.06562853341467},
 'SVR': {'MAE': 18.299609026628282,
  'MSE': 1964.7535818311444,
  'MSE_std': 302.1950027931552},
 'Random Forest': {'MAE': 5.788337364748086,
  'MSE': 195.58115266122235,
  'MSE_std': 22.496872929381738},
 'Gradient Boosting': {'MAE': 8.82080060042972,
  'MSE': 389.24053595820317,
  'MSE_std': 32.33608498184415},
 'Artificial Neural Network': {'MAE': 12.523924398825951,
  'MSE': 643.1438894348701,
  'MSE_std': 84.2561701392796}}

In [45]:
# after hyperparametertuning
results

In [44]:
import pickle
with open('result_cutout_after_featureselection(wo gpr).pkl', 'wb') as file:
    pickle.dump(results_feature, file)


{'Linear Regression': {'MAE': 19.000541966353587, 'MSE': 1350.56069460534},
 'Lasso': {'MAE': 22.610703312854127, 'MSE': 1679.4760262080995},
 'Ridge': {'MAE': 18.989592790164192, 'MSE': 1349.7480663335148},
 'SVR': {'MAE': 23.793351948629688, 'MSE': 1756.1078902638878},
 'Random Forest': {'MAE': 17.635830260141653, 'MSE': 2015.2344103327919},
 'Gradient Boosting': {'MAE': 15.969274342418169, 'MSE': 1596.9989870917648},
 'Artificial Neural Network': {'MAE': 21.345942437519778,
  'MSE': 1888.5103715269568}}

In [45]:
results_feature

{'Linear Regression': {'MAE': 19.47148762371669,
  'MSE': 874.7590684153923,
  'selected_feature': Index(['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes',
         'start_node_x', 'start_node_y', 'end_node_x', 'end_node_y',
         'start_count'],
        dtype='object')},
 'Lasso': {'MAE': 21.29080224423866,
  'MSE': 995.8290100956449,
  'selected_feature': Index(['link_id', 'link_permlanes', 'start_node_x', 'start_node_y',
         'end_node_x', 'end_node_y', 'start_count', 'end_count', 'go_to_sum'],
        dtype='object')},
 'Ridge': {'MAE': 18.855381894067843,
  'MSE': 831.7957770846882,
  'selected_feature': Index(['link_length', 'link_freespeed', 'link_capacity', 'link_permlanes',
         'start_node_x', 'start_node_y', 'end_node_y', 'start_count',
         'end_count'],
        dtype='object')},
 'SVR': {'MAE': 21.113999718501606,
  'MSE': 998.1921690104024,
  'selected_feature': Index(['link_id', 'link_length', 'link_freespeed', 'link_capacity',
         '

In [8]:
param_grid_svr = {
    'C': [0.01, 0.1, 1, 10],  # Extended range for the regularization parameter
    'gamma': ['scale', 'auto'],  # Including specific gamma values
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Focusing on RBF kernel
    'epsilon': [0.01, 0.1, 0.2],  # Epsilon in the epsilon-SVR model
}

mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
random_search_svr = GridSearchCV(SVR(max_iter=2000), param_grid_svr, cv=kf, n_jobs=-1, verbose=10, scoring=mse_scorer)
random_search_svr.fit(X_t, y_t)
print(random_search_svr.best_params_)
print(random_search_svr.best_estimator_)
print(random_search_svr.best_score_)

Fitting 3 folds for each of 70 candidates, totalling 210 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [-2.06753800e+21 -1.78347029e+03 -3.21274664e+21 -4.52036712e+58
 -2.11726699e+03 -1.78346984e+03 -1.78347029e+03 -1.78347029e+03
             nan             nan -6.24658420e+19 -6.10801167e+12
 -1.96408398e+16 -1.78347029e+03             nan -1.78347029e+03
 -1.78347029e+03 -5.64219848e+20             nan -5.87544447e+21
             nan -1.78347029e+03 -1.78347029e+03             nan
 -1.78347029e+03             nan -4.64578815e+15 -1.77140788e+03
             nan -1.78347029e+03 -1.18411241e+16 -1.92220132e+03
 -1.78347029e+03 -1.78346995e+03             nan -2.11002684e+03
 -1.07400560e+17             nan             nan -1.78347029e+03
 -2.04274595e+03 -1.78347029e+03 -1.78347028e+03 -1.78347029e+03
 -1.78347029e+03 -1.78346908e+03 -1.77144765e+03 -1.78204420e+03
 -1.78347029e+03 -1.78347029e+03 -1.78347029e+03 -1.78070475e+03
 -2.11794703e

{'C': 25.383309585489613, 'epsilon': 0.01, 'gamma': 2.1969677491639246, 'kernel': 'rbf'}
SVR(C=25.383309585489613, epsilon=0.01, gamma=2.1969677491639246, max_iter=2000)


C:\Users\user\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [66]:
param_grid_rf = {
    'n_estimators': [100, 150, 200, 250],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 4],
    'min_samples_leaf': [1, 2],
    'criterion':['friedman_mse']
}
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search_rf = GridSearchCV(RandomForestRegressor(random_state=42), param_grid_rf, cv=kf, n_jobs=-1, verbose=10, scoring=mse_scorer)
grid_search_rf.fit(X_t, y_t)

print(grid_search_rf.best_params_)
print(grid_search_rf.best_estimator_)
print(grid_search_rf.best_score_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
{'criterion': 'friedman_mse', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
RandomForestRegressor(criterion='friedman_mse', n_estimators=200,
                      random_state=42)
-194.422856040387


In [ ]:
param_grid_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],  # Varied learning rates for gradient boosting
    'max_depth': [3, 5, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 4],
    'subsample': [0.8, 1.0],  # Fraction of samples to be used for fitting individual base learners
}

mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search_gb = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid_gb, cv=kf, n_jobs=-1, verbose=10, scoring=mse_scorer)
grid_search_gb.fit(X_t, y_t)
print(grid_search_gb.best_params_)
print(grid_search_gb.best_estimator_)
print(grid_search_gb.best_score_)

In [ ]:
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 100), (30, 30, 30)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
}
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search_mlp = GridSearchCV(MLPRegressor(max_iter=2000, random_state=42), param_grid_mlp, cv=kf, n_jobs=-1, verbose=10, scoring=mse_scorer)
grid_search_mlp.fit(X_t, y_t)
print(grid_search_mlp.best_params_)
print(grid_search_mlp.best_estimator_)
print(grid_search_mlp.best_score_)

In [35]:
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, DotProduct
import numpy as np
param_grid = {
    'kernel': [ConstantKernel (1.0, (1e-1, 1e1)) * RBF(1.0, (1e-2, 1e2))],
    'alpha': [ 1e-2, 0.1, 1.0]
}

gpr = GaussianProcessRegressor(copy_X_train=False)

# Initialize GridSearchCV
grid_search_gpr = RandomizedSearchCV(gpr, param_grid, n_iter=5, cv=0, scoring='neg_mean_squared_error', n_jobs=-1, verbose=10)
grid_search_gpr.fit(X_t, y_t)

print(grid_search_gpr.best_params_)
print(grid_search_gpr.best_estimator_)

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=0.

In [59]:
def evaluate_models_with_test(model, X_train, y_train, X_test, y_test):
    results = {}
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    # mape = mean_absolute_percentage_error(y_test, y_pred)
    # r2 = r2_score(y_test, y_pred)
      
    results = {'MAE': mae, 'MSE': mse}
    
    return results

In [ ]:
import pickle

result_final_with_test = {}
for name, model in models.items():
    X_t_reduced = X_t[results_feature[name]['selected_feature']]
    X_te_reduced = ln_X_te = X_te[results_feature[name]['selected_feature']]
    result_final_with_test[name] = evaluate_models_with_test(model, X_t_reduced, y_t, X_te_reduced, y_te)

with open('result_cutout_final(wo gpr).pkl', 'wb') as file:
    pickle.dump(result_final_with_test, file)

In [71]:
ln_results

{'MAE': 20.1076673770882, 'MSE': 902.1385041302448}

In [72]:
lasso_results

{'MAE': 14.695032409248478, 'MSE': 570.2595372331552}

In [73]:
ridge_results

{'MAE': 20.91790633288539, 'MSE': 955.1395825360881}

In [74]:
svr_results

{'MAE': 23.39522142225316, 'MSE': 1003.343949315722}

In [60]:
rf_results

{'MAE': 14.532470756135266, 'MSE': 676.9930176271231}

In [64]:
gb_results

{'MAE': 17.260081814945043, 'MSE': 721.9088243448939}

In [65]:
ann_results

{'MAE': 22.55849174892375, 'MSE': 1005.3902663530097}

In [ ]:
# # Initialize a list to hold trips
# trips = []
# current_trip = [df_od_pairs.iloc[0]['origin']]  # Start with the first origin
# 
# # Iterate over the DataFrame rows
# for i, row in df_od_pairs.iterrows():
#     current_trip.append(row['destination'])  # Always add the destination
#     # Check if the next origin matches the current destination
#     if i + 1 < len(df_od_pairs) and row['destination'] != df_od_pairs.iloc[i + 1]['origin']:
#         # If it doesn't, the current trip has ended
#         trips.append(current_trip)
#         current_trip = [df_od_pairs.iloc[i + 1]['origin']]  # Start a new trip
# 
# # Add the last trip if it wasn't already added
# if current_trip not in trips:
#     trips.append(current_trip)


# from collections import Counter
# # Flatten the list of trips into a single list of nodes including origins and destinations
# all_nodes = [node for trip in trips for node in trip]
# 
# # Use Counter to count the occurrences of each node
# node_trip_counts = Counter(all_nodes)
# 
# df_node_trip_counts = pd.DataFrame.from_dict(node_trip_counts, orient='index').reset_index()
# df_node_trip_counts.columns = ['node_id', 'trip_amount']